#### Import the necessary libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
print(torch.__version__)
from torch.utils.data import DataLoader, TensorDataset

c:\Users\ibrah\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.5.1+cpu


#### Load train data


In [2]:
train_df = pd.read_json('Data Samples/train.json')
print(train_df.shape)
train_df.head()

(7401, 2)


,reviews,sentiments
0,I bought this belt for my daughter in-law for ...,1
1,The size was perfect and so was the color. It...,1
2,"Fits and feels good, esp. for doing a swim rac...",1
3,These socks are absolutely the best. I take pi...,1
4,Thank you so much for the speedy delivery they...,1


### We first need to tokenize the reviews and prepare the data for BERT. 
### BERT expects tokenized input in the form of input IDs, attention masks, and possibly token type IDs.

In [3]:
# Instantiate the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocess the reviews
def encode_reviews(reviews, tokenizer, max_length=256):
    return tokenizer.batch_encode_plus(
        reviews, 
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        return_attention_mask=True,  # Return attention masks
        padding='max_length',  # Pad to max_length
        truncation=True,  # Truncate longer sequences
        max_length=max_length,  # Maximum length of sequences
        return_tensors='pt'  # Return PyTorch tensors
    )

In [4]:
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = ["Hello, how are you?"]

# Tokenize and check if PyTorch tensors are returned
encoding = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
print(encoding)


{'input_ids': tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


#### Encode reviews

In [5]:
train_encodings = encode_reviews(train_df['reviews'].tolist(), tokenizer)

#### You can use the TfidfVectorizer from the scikit-learn library to convert the text to vectors

The vectorizer should only be fitted on the training data as the test data are unknown during training. 

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(train_df['reviews'])
X_train = vectorizer.transform(train_df['reviews'])
X_train.shape

(7401, 14114)

#### Test set should be transformed using the same vectorizer (fitted on the training data)

In [ ]:
test_df = pd.read_json('Data Samples/test.json')
print(test_df.shape)
X_test = vectorizer.transform(test_df['reviews'])
print(X_test.shape)

(1851, 1)
(1851, 14114)
